In [ ]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ../content/drive/MyDrive/'Spotify Project InfoVis'/

/content/drive/MyDrive/Spotify Project InfoVis


In [ ]:
df_open = pd.read_csv('./spotify_dataset.csv')

In [ ]:
relevant_cols = df_open[['Artist','Number of Times Charted', 'Release Date', 'Genre']]
relevant_cols['Release Date'] = pd.to_datetime(relevant_cols['Release Date'], errors='coerce')
relevant_cols = relevant_cols[pd.notnull(relevant_cols['Release Date'])]
relevant_cols = relevant_cols.loc[relevant_cols['Release Date'].dt.year >= 2012]
relevant_cols['date'] = relevant_cols['Release Date'].apply(lambda s: dt.date(s.year, s.month, 1))
df = relevant_cols.drop(columns = {'Release Date'})

df = df.rename(columns={'Number of Times Charted':'num_chart'})

<ipython-input-52-72a07af9a380>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_cols['Release Date'] = pd.to_datetime(relevant_cols['Release Date'], errors='coerce')


In [ ]:
artists = df['Artist'].tolist()
genres = df['Genre'].tolist()

In [ ]:
all_artists = ','.join(map(str,artists)).split(',')
all_artists = set([x.strip() for x in all_artists])

all_genres = ','.join(map(str,genres)).split(',')
all_genres = set([x.lstrip("[").rstrip("]") for x in all_genres])
all_genres = set([x.strip(" ") for x in all_genres])
all_genres = set([x.lstrip("'").rstrip("'") for x in all_genres])
all_genres = set([x.lstrip("'").rstrip("'") for x in all_genres])

In [ ]:

charted_dict = dict.fromkeys(all_artists, 0)
genre_dict = dict.fromkeys(all_genres,0)

dict_artists = list()
dict_genres = list()

start = dt.date(2012,1,1)
end = dt.date(2021,8,1)

while start <= end:
  df_year = df.loc[df.date == start]

  for index, row in df_year.iterrows():
    cur_artist = [x.strip() for x in row['Artist'].split(',')]

    cur_genre = row['Genre'].split(',')
    cur_genre = [x.lstrip("[").rstrip("]") for x in cur_genre]
    cur_genre = [x.strip(" ") for x in cur_genre]
    cur_genre = [x.lstrip("'").rstrip("'") for x in cur_genre]
    cur_genre = [x.lstrip("'").rstrip("'") for x in cur_genre]

    for a in cur_artist:
      charted_dict[a] += row['num_chart']

    for a in cur_genre:
      genre_dict[a] += row['num_chart']

  copy_dict = copy.deepcopy(charted_dict)
  copy_genres = copy.deepcopy(genre_dict)

  for k in copy_dict:
    dict_artists.append({'date': start,
                          'artist': k,
                          'num_charted': copy_dict[k]})
  for k in copy_genres:
    dict_genres.append({'date': start,
                          'genre': k,
                          'num_charted': copy_genres[k]})

  start += relativedelta(months = 1)

In [ ]:
dict_genres

[{'date': datetime.date(2012, 1, 1), 'genre': '', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1),
  'genre': 'canadian hip hop',
  'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'shiver pop', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1),
  'genre': 'electronic trap',
  'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'pop edm', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'drill', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'trap triste', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'uk dance', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'viral rap', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'boy band', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1),
  'genre': 'nouvelle chanson francaise',
  'num_charted': 0},
 {'date': datetime.date(2012, 1, 1), 'genre': 'europop', 'num_charted': 0},
 {'date': datetime.date(2012, 1, 1),
 

In [ ]:
import csv
csv_columns = ['date','artist','num_charted']
with open('./spotify_data_artists.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in dict_artists:
        writer.writerow(data)

In [ ]:
csv_columns = ['date','genre','num_charted']
with open('./spotify_data_genres.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in dict_genres:
        writer.writerow(data)